<a href="https://colab.research.google.com/github/gustavomaximo82/Projeto_Imers-oIA_Alura-Google/blob/main/Projecto_imers%C3%A3o_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U google-generativeai
!pip install PyPDF2

In [2]:
from pathlib import Path
import PyPDF2
import os

# Import the Python SDK
import google.generativeai as genai
from google.colab import userdata
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key = api_key)

In [3]:
#Instalando o SDK do Google
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [4]:
# Configuração do modelo
generation_config = {
  "candidate_count": 1,
  "temperature": 0.5,
}

safety_settings={
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
    }

In [5]:
# inicializando o modelo
model = genai.GenerativeModel(model_name='gemini-1.0-pro',
                                  generation_config=generation_config,
                                  safety_settings=safety_settings,)

In [6]:
# Função para ler arquivos pdf no diretório
def list_files(directory):
    file_list = []
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if os.path.isfile(file_path):
            file_list.append(file_path)
    return file_list

In [7]:
# Validação aruivos identificados
arquivos = list_files("/content/drive/MyDrive/ImersaoIA")
print(arquivos)

['/content/drive/MyDrive/ImersaoIA/NF1.pdf', '/content/drive/MyDrive/ImersaoIA/NF2.pdf', '/content/drive/MyDrive/ImersaoIA/NF3.pdf']


In [8]:
# Função para extrair o texto de um arquivo PDF.
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        text = ""
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    return text

In [9]:
# Inicia um chat com o modelo.
chat = model.start_chat(history=[])

In [10]:
# Extrai o texto do arquivo PDF.
pdf_text = []
for file in arquivos:
  pdf_text.append(extract_text_from_pdf(file))

numero_de_NFs = len(pdf_text)

In [11]:
# Envia o texto do PDF para o modelo e obtém a resposta.
for i in range(numero_de_NFs):
  response = chat.send_message(f'Nota fiscal {i+1}: {pdf_text[i]}')
  print(f'Nota fiscal {i+1} carregada')

Nota fiscal 1 carregada
Nota fiscal 2 carregada
Nota fiscal 3 carregada


In [12]:
# Calibração do modelo
response = chat.send_message('Desconsidere os primeiros valores de data e hora encontrada no texto, que vem logo após o termo "Nota Fiscal". Busque o próximo valor de data e hora e classifique-o como data de emissão da nota fiscal')
response = chat.send_message('Liste as notas ficais com as respectivas datas de emissão e ordene-as da mais antiga para a mais recente.')
print(response.text)

**Notas fiscais ordenadas da mais antiga para a mais recente:**

1. **Nota fiscal 1:**
    * Data de emissão: 06/02/2024 19:26:34
2. **Nota fiscal 2:**
    * Data de emissão: 11/02/2024 19:15:34
3. **Nota fiscal 3:**
    * Data de emissão: 29/03/2024 14:07:08


In [13]:
# Calibração do modelo
response = chat.send_message('Utilize sempre a seguinte lógica para validar seu modelo e garantir que você toma os valores unitários corretamente: Qtde x Vl. Unit. = Vl. Total')

In [14]:
# Inicia um loop que continua até que o usuário digite "fim".
prompt = input('Esperando prompt: ')
while prompt != "fim":
  # Envia o prompt do usuário para o modelo e obtém a resposta.
  response = chat.send_message(prompt)

  # Imprime a resposta do modelo.
  print("## Resposta:\n\n" + response.text + '\n\n')

  # Solicita um novo prompt do usuário.
  prompt = input('Esperando prompt: ')
  print("_____________________________________________________________")

Esperando prompt: Organize as informações de cada uma das notas fiscais em uma tabela com o seguinte formato: Coluna 1 = Nome do supermercado Coluna 2 = Data da emissão Coluna 3 = Valor total da nota fiscal Considere no novo ordenamentodas notas fiscais e a instrução aprendida sobre as datas de emissão.
## Resposta:

| Nome do supermercado | Data da emissão | Valor total da nota fiscal |
|---|---|---|
| SENDAS DISTRIBUIDORA S/A | 29/03/2024 | R$ 1.999,75 |
| ANGELONI CIA LTDA | 06/02/2024 | R$ 525,66 |
| SENDAS DISTRIBUIDORA S/A | 11/02/2024 | R$ 1.541,99 |


Esperando prompt: Organize a informação de cada uma das notas fiscais em tabelas separadas. As tabelas devem ter todas o seguinte formato: Coluna 1 = Nome do supermercado Coluna 2 = Data de emissão Coluna 3 = Produto Coluna 4 = Qtde Coluna 5 = Vl. Unit. Coluna 6 = UN Coluna 7 = Vl. Total
_____________________________________________________________
## Resposta:

**Nota fiscal 1**

| Nome do supermercado | Data de emissão | Produto